<a href="https://colab.research.google.com/github/pablex72/tensorflow-practices/blob/main/RNN_keras_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importacion de librerias**

In [1]:
import numpy as np
np.random.seed(5)

In [2]:
from keras.layers import Input, Dense, SimpleRNN
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import backend as K
# “to_categorical” y el backend para poder representar la entrada y salida 
# al modelo en el formato adecuado durante el entrenamiento y la durante la predicción

**Lectura y pre-procesamiento del set de datos**

In [4]:
nombres = open('/content/nombres_dinosaurios.txt','r').read()
nombres = nombres.lower()
nombres

'aachenosaurus\naardonyx\nabdallahsaurus\nabelisaurus\nabrictosaurus\nabrosaurus\nabydosaurus\nacanthopholis\nachelousaurus\nacheroraptor\nachillesaurus\nachillobator\nacristavus\nacrocanthosaurus\nacrotholus\nactiosaurus\nadamantisaurus\nadasaurus\nadelolophus\nadeopapposaurus\naegyptosaurus\naeolosaurus\naepisaurus\naepyornithomimus\naerosteon\naetonyxafromimus\nafrovenator\nagathaumas\naggiosaurus\nagilisaurus\nagnosphitys\nagrosaurus\nagujaceratops\nagustinia\nahshislepelta\nairakoraptor\najancingenia\najkaceratops\nalamosaurus\nalaskacephale\nalbalophosaurus\nalbertaceratops\nalbertadromeus\nalbertavenator\nalbertonykus\nalbertosaurus\nalbinykus\nalbisaurus\nalcovasaurus\nalectrosaurus\naletopelta\nalgoasaurus\nalioramus\naliwalia\nallosaurus\nalmas\nalnashetri\nalocodon\naltirhinus\naltispinax\nalvarezsaurus\nalwalkeria\nalxasaurus\namargasaurus\namargastegos\namargatitanis\namazonsaurus\nammosaurus\nampelosaurus\namphicoelias\namphicoelicaudia\namphisaurus\namtocephale\namtosaur

Como la Red Recurrente no acepta caracteres a la entrada, debemos convertir cada uno de estos a una representación numérica.

Para ello definiremos un alfabeto, que corresponde a los diferentes caracteres que conforman el set de datos.

In [5]:
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)

cada carácter será representado en el formato one-hot, es decir con un vector de 27 elementos

In [6]:
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}


convertir vectores one-hot a su carácter correspondiente, para lo cual crearemos un segundo diccionario que permite mapear índices a caracteres:

In [9]:
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}

**Creación de la Red Recurrente en Keras**


En primer lugar definimos el número de neuronas de la capa oculta (es decir 25), y creamos dos contenedores:

In [10]:
n_a = 25    # Número de unidades en la capa oculta

y creamos dos contenedores:

Uno para la entrada ( en el diagrama anterior), en donde hemos usado la palabra clave None, lo cual nos permite tener entradas (es decir nombres de dinosaurio) con tamaños variables. 

In [11]:
entrada  = Input(shape=(None,tam_alfabeto))

Uno para el estado oculto ( en el diagrama anterior), que será precisamente un vector con 25 elementos:

In [12]:
a0 = Input(shape=(n_a,))

Ahora creamos la celda recurrente. Para ello usamos la función SimpleRNN importada anteriormente.


Esta celda tendrá 25 neuronas, función de activación tangente hiperbólica y usaremos la palabra clave return_state para que a la salida nos entregue el nuevo estado oculto actualizado:



In [13]:
celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True)

Ahora creamos la capa softmax. Para ello usamos la función Dense. Esta capa tendrá 27 neuronas de salida (el tamaño del alfabeto) y usará precisamente la función de activación “softmax”:



In [14]:
capa_salida = Dense(tam_alfabeto, activation='softmax')

en Keras, usando la función Model importada anteriormente.

In [15]:
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida = []
salida.append(capa_salida(hs))

Ahora creamos el modelo, dejando explícito que tendremos dos entradas (el caracter actual y el estado oculto anterior) y la salida correspondiente a la predicción:

In [16]:
modelo = Model([entrada,a0],salida)

In [17]:
opt = SGD(lr=0.0005)
modelo.compile(optimizer=opt, loss='categorical_crossentropy')

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


**Entrenamiento de la Red Recurrente**

In [18]:
with open("nombres_dinosaurios.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

Después definimos una función que tome uno a uno cada ejemplo de entrenamiento y que genere tres vectores, que serán las entradas al modelo:

In [22]:
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y

Finalmente definimos la activación inicial que será simplemente una matriz de ceros, y que se irá actualizando a medida que se presentan uno a uno los caracteres al modelo durante el entrenamiento:


**ENTRENAMIENTO**

In [ ]:
BATCH_SIZE = 80			# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 10000			# Número de iteraciones

for j in range(NITS):
    historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    if j%1000 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')

<ipython-input-24-8076776f9f0c>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)



Iteración: 0, Error: 3.282617


Iteración: 1000, Error: 2.380164


Iteración: 2000, Error: 2.249877


Iteración: 3000, Error: 2.169339


Iteración: 4000, Error: 2.188374


Iteración: 5000, Error: 2.153284


Iteración: 6000, Error: 2.138089


Iteración: 7000, Error: 2.231277


Iteración: 8000, Error: 2.181410


Iteración: 9000, Error: 2.169926



**Predicción con la Red Recurrente: generación de nombres de dinosaurios**

In [ ]:
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          y = capa_salida(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 50):
            nombre_generado += '\n'

    print(nombre_generado)


Generar 100 ejemplos de nombres generados por el modelo ya entrenado


In [ ]:
for i in range(100):
    generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)